**Notebook to model electricity price differences.**<br>

Brief summary of the exercise:
* Target: predict the price change of electricity from the Close price of one hour to the Close price of the next hour
* Input: the elementwise difference between the hourly 'candels' of the electricity price; explanation follows.

  Hourly candels refers to the Open, High, Low, Close prices and the total Volume traded within an hour; thus each hourly candel has five elements. Historically, one could reconstruct these candels for each consecutive hour. We refer to the length of history for which the candels are reconstructed as 'window_size'. Thus, for a window_size of 5, one has candels from 5 consecutive hours.
  
  Left to explain is the 'elementwise difference' remark. Out of the N window_size of history, we compute differences between the consecutive candels: Open in t minus Open in t-1, Low in t minus low in t-1, etc. Thus, from N candels we create N-1 differentiated candels.

  In this exercise, the elementwise differentiated data for window_sizes of 5 and 15 are provided

  The data is further enchanced by mostly encoded time-related information. These encode what day of the week the data correspond to, whether the day is a bank holiday or not, etc. A list will be given further below.

* ML models are to be built to predict the Target from the Input data.

In [1]:
# Load the data using the pandas library
# For the below demonstration, only the
#    * window_size of 5 files
#    * and only the training sets will be loaded here
# The validation and test sets can be similarly loaded.
# One can also use a similar kind of method for loading as was used for the download
# This can avoid the many lines of manual specification of the files. Manual is also fine, btw.

import pandas as pd
pd.set_option('display.max_columns', None)

X_train_5 = pd.read_csv('X_train_window_size_5_time_encoding_True.csv')
y_train_5 = pd.read_csv('y_train_window_size_5_time_encoding_True.csv')

# Display the first 5 rows of the predictor and the target data; for a description of the content, 
# see the text below this cell
print("Predictor data:")
display(X_train_5.head(10))
print("Target data:")
display(y_train_5.head())

Predictor data:


,total_hours,dlvry_weekend,dlvry_bank_holiday,dlvry_day_sin,dlvry_day_cos,dlvry_weekday_sin,dlvry_weekday_cos,dlvry_hour_sin,dlvry_hour_cos,lasttrade_weekend,lasttrade_bank_holiday,lasttrade_day_sin,lasttrade_day_cos,lasttrade_weekday_sin,lasttrade_weekday_cos,lasttrade_hour_sin,lasttrade_hour_cos,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.62349,-0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.707107,-7.071068e-01,0.03,0.80,-1.17,-1.18,92.2,600.0,-9.51,-1.98,-8.31,-2.83,176.6,540.0,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0
1,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.62349,-0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.866025,-5.000000e-01,-9.51,-1.98,-8.31,-2.83,176.6,540.0,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0
2,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.62349,-0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.965926,-2.588190e-01,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0
3,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.62349,-0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-1.000000,-1.836970e-16,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0,-2.90,-1.69,2.44,-0.01,-355.6,240.0
4,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.62349,-0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.965926,2.588190e-01,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0,-2.90,-1.69,2.44,-0.01,-355.6,240.0,-0.70,-1.11,-1.11,-2.69,53.3,180.0
5,7.080482,0.0,0.0,0.394356,0.918958,0.000000,1.00000,0.258819,0.965926,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.707107,7.071068e-01,0.00,2.99,0.00,0.00,20.0,420.0,1.77,-1.20,0.00,0.50,40.6,360.0,-1.59,-1.09,-2.65,-2.21,555.0,300.0,-1.90,1.10,0.38,1.61,-332.7,240.0
6,7.080482,0.0,0.0,0.394356,0.918958,0.000000,1.00000,0.258819,0.965926,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.500000,8.660254e-01,1.77,-1.20,0.00,0.50,40.6,360.0,-1.59,-1.09,-2.65,-2.21,555.0,300.0,-1.90,1.10,0.38,1.61,-332.7,240.0,1.94,-1.58,-2.42,-2.26,421.6,180.0
7,7.726036,0.0,0.0,0.394356,0.918958,0.000000,1.00000,0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.500000,8.660254e-01,-0.21,-1.00,1.26,1.26,23.1,420.0,-1.69,0.07,-0.86,-0.36,328.4,360.0,-1.39,0.63,-0.70,2.23,-86.4,300.0,3.02,-0.40,-1.93,-2.61,377.2,240.0
8,7.726036,0.0,0.0,0.394356,0.918958,0.000000,1.00000,0.500000,0.866025,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.258819,9.659258e-01,-1.69,0.07,-0.86,-0.36,328.4,360.0,-1.39,0.63,-0.70,2.23,-86.4,300.0,3.02,-0.40,-1.93,-2.61,377.2,240.0,-2.61,-1.70,0.23,0.49,104.2,180.0
9,8.726036,0.0,0.0,0.394356,0.918958,0.000000,1.00000,0.707107,0.707107,1.0,0.0,0.201299,0.97953,-0.781831,0.62349,-0.500000,8.660254e-01,-0.60,-1.21,-3.48,-4.60,13.0,480.0,-4.08,-1.60,-3.20,-3.20,175.2,420.0,-0.01,5.01,1.99,7.16,108.5,360.0,5.08,-2.93,-0.99,-3.26,121.3,300.0


Target data:


,y
0,-3.18
1,-3.21
2,-0.01
3,-2.69
4,2.41


In [2]:
X_test_5 = pd.read_csv('X_test_window_size_5_time_encoding_True.csv')
y_test_5 = pd.read_csv('y_test_window_size_5_time_encoding_True.csv')

X_valid_5 = pd.read_csv('X_valid_window_size_5_time_encoding_True.csv')
y_valid_5 = pd.read_csv('y_valid_window_size_5_time_encoding_True.csv')

In [3]:
from sklearn.metrics import mean_squared_error 
import numpy as np

def evaluate_model(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    mse = mean_squared_error(y_valid_true, predictions)
    print("Mean squared error on valid:",mse)
    return mse

# Modified from assignment 6
def evaluate_model_2(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    mse = mean_squared_error(y_valid_true, predictions)
    print("Mean squared error on valid:",mse)
    normalized_mse = normalizers["y"].inverse_transform(np.array([mse]).reshape(1, -1))[0][0]
    print("Mean squared error on valid inverse transformed from normalization:",normalized_mse)
    return normalized_mse

In [4]:
X_train_5

,total_hours,dlvry_weekend,dlvry_bank_holiday,dlvry_day_sin,dlvry_day_cos,dlvry_weekday_sin,dlvry_weekday_cos,dlvry_hour_sin,dlvry_hour_cos,lasttrade_weekend,lasttrade_bank_holiday,lasttrade_day_sin,lasttrade_day_cos,lasttrade_weekday_sin,lasttrade_weekday_cos,lasttrade_hour_sin,lasttrade_hour_cos,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.500000,0.866025,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.707107,-7.071068e-01,0.03,0.80,-1.17,-1.18,92.2,600.0,-9.51,-1.98,-8.31,-2.83,176.6,540.0,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0
1,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.500000,0.866025,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.866025,-5.000000e-01,-9.51,-1.98,-8.31,-2.83,176.6,540.0,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0
2,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.500000,0.866025,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.965926,-2.588190e-01,5.48,2.58,5.39,5.41,187.7,480.0,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0
3,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.500000,0.866025,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-1.000000,-1.836970e-16,4.41,-0.21,1.30,-1.31,-18.2,420.0,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0,-2.90,-1.69,2.44,-0.01,-355.6,240.0
4,10.001383,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.500000,0.866025,1.0,0.0,0.201299,0.979530,-0.781831,0.623490,-0.965926,2.588190e-01,-0.31,-0.20,-0.66,-3.18,-78.3,360.0,-3.49,-2.59,-7.97,-3.21,796.3,300.0,-2.90,-1.69,2.44,-0.01,-355.6,240.0,-0.70,-1.11,-1.11,-2.69,53.3,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91507,17.591981,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.965926,-0.258819,0.0,0.0,-0.587785,-0.809017,-0.433884,-0.900969,-0.258819,9.659258e-01,-1.27,-2.17,-0.91,1.04,0.9,660.0,0.11,1.43,2.18,0.70,77.8,600.0,0.59,0.55,0.42,0.85,19.8,540.0,2.10,5.12,2.20,6.74,-3.2,480.0
91508,17.591981,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.965926,-0.258819,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.000000,1.000000e+00,0.11,1.43,2.18,0.70,77.8,600.0,0.59,0.55,0.42,0.85,19.8,540.0,2.10,5.12,2.20,6.74,-3.2,480.0,6.38,5.64,6.58,4.05,233.7,420.0
91509,17.591981,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.965926,-0.258819,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.258819,9.659258e-01,0.59,0.55,0.42,0.85,19.8,540.0,2.10,5.12,2.20,6.74,-3.2,480.0,6.38,5.64,6.58,4.05,233.7,420.0,2.76,1.24,1.88,-0.41,-170.3,360.0
91510,17.591981,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.965926,-0.258819,1.0,0.0,-0.743145,-0.669131,-0.974928,-0.222521,0.500000,8.660254e-01,2.10,5.12,2.20,6.74,-3.2,480.0,6.38,5.64,6.58,4.05,233.7,420.0,2.76,1.24,1.88,-0.41,-170.3,360.0,-0.83,-1.00,-0.25,-0.13,-81.8,300.0


In [7]:
from sklearn.ensemble import RandomForestRegressor

N_ESTIMATORS = 20
RANDOM_STATE = 452543634
RF_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, n_jobs=4)

RF_model.fit(X_train_5, y_train_5)

RandomForestRegressor(n_estimators=20, n_jobs=4, random_state=452543634)

In [8]:
result = evaluate_model_2(RF_model, X_valid_5, y_valid_5)

Mean squared error on valid: 0.00045966155038540103
Mean squared error on valid inverse transformed from normalization: -74.14704736722378


### Simple LSTM

In [5]:
# LSTM_CELL_SIZE=64
BATCH_SIZE = 200
EPOCHS = 15
# DROPOUT_RATE=0.15

#### Create an artificial 3rd dimension

In [6]:
X_train_5 = X_train_5.to_numpy().reshape(X_train_5.shape[0], 1, X_train_5.shape[1])
X_valid_5 = X_valid_5.to_numpy().reshape(X_valid_5.shape[0], 1, X_valid_5.shape[1])

In [8]:
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

column_count = np.shape(X_train_5)[2]

be.clear_session()

# def schedule(epoch, lr):
#    ...
#    return lr

# lr_scheduler = LearningRateScheduler(schedule)

es = EarlyStopping(monitor='val_loss', mode='min', 
                   verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', 
                     mode='min', save_best_only=True)

# Build your whole LSTM model here!
model = Sequential()
model.add(LSTM(100, input_shape=(1, column_count), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(120))
# model.add(Bidirectional(LSTM(150)))
model.add(Dropout(0.4))
model.add(Dense(1))

#For shape remeber, we have a variable defining the "window" and the features in the window...
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Fit on the train data
# USE the batch size parameter!
# Use validation data - warning, a tuple of stuff!
# Epochs as deemed necessary...
# You should avoid shuffling the data maybe.
# You can use the callbacks for LR schedule or model saving as seems fit.

history = model.fit(X_train_5, y=y_train_5,
                    validation_data=(X_valid_5, y_valid_5),
                    shuffle=False,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=[mc, es])

Epoch 1/15
458/458 [==============================] - 6s 7ms/step - loss: 11.9224 - val_loss: 25.6901
Epoch 2/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9140 - val_loss: 25.6945
Epoch 3/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9124 - val_loss: 25.6879
Epoch 4/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9064 - val_loss: 25.7005
Epoch 5/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9092 - val_loss: 25.6856
Epoch 6/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9072 - val_loss: 25.6961
Epoch 7/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9043 - val_loss: 25.6850
Epoch 8/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9048 - val_loss: 25.7005
Epoch 9/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9012 - val_loss: 25.6850
Epoch 10/15
458/458 [==============================] - 3s 6ms/step - loss: 11.9051

In [10]:
result = evaluate_model(model, X_valid_5, y_valid_5)

Mean squared error on valid: 25.69197244896494
